<a href="https://colab.research.google.com/github/chungchihhan/https-github.com-kszuyen-A-Dating-App-for-Dogs/blob/main/llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing packages

In [13]:
!pip install llama-index

In [41]:
!pip install replicate

In [14]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

Get the data from google drive

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os

# Google 雲端硬碟的根目錄路徑
path = "/content/drive/My Drive"

# 列出指定路徑下的所有檔案和資料夾
# files = os.listdir(path)
# for file in files:
#     print(file)


In [20]:
filepath = '/content/drive/MyDrive/TS31103_Adjustment_of_Backlash_for_GXA_S_Series_converted.txt'

Set up the api key

In [34]:
# Set up the openai_api_key on the left sidebar, there's a place to input secret there.

# <<Alternative way>> :
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

from google.colab import userdata
# userdata.get('OPENAI_API_KEY')

 Build a simple vector store index using OpenAI:

In [35]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=[filepath]
)
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")
# documents = SimpleDirectoryReader(filepath).load_data()
# index = VectorStoreIndex.from_documents(documents)

index = VectorStoreIndex.from_documents(docs)

Loaded 1 docs


In [36]:
query_engine = index.as_query_engine()

result = query_engine.query("背隙調整步驟")
response_text = result.response

print(response_text)

背隙調整步驟包括以下幾個步驟：
1. 在套管座上鎖緊M6x50L螺絲2只。
2. 用這2只螺絲旋轉套管座。順時針方向為縮小蝸桿蝸輪背隙，逆時針方向為放大蝸桿蝸輪背隙。建議一次最多轉動1/4圈。
3. 使用T型板手2.5mm拆開皿型螺絲4只，將軸封蓋取下。
4. 調整完背隙後，轉動馬達直到T型扳手4mm可以套入聯軸器上的M5螺絲②，然後將螺絲鎖緊。
5. 依序將塞蓋以及蝸桿封蓋裝回，即完成蝸桿蝸輪的背隙調整。


Build a simple vector store index using non-OpenAI LLMs ,e.g. Llama 2 hosted on [Replicate](https://replicate.com/), where you can easily create a free trial API token:

In [61]:
# Set up the openai_api_key on the left sidebar, there's a place to input secret there.

# <<Alternative way>> :
# import os
# os.environ["REPLICATE_API_TOKEN"] = "YOUR_REPLICATE_API_TOKEN"

from google.colab import userdata
# userdata.get('REPLICATE_API_TOKEN')

In [56]:
from llama_index.llms import Replicate

llama2_7b_chat = "meta/llama-2-7b-chat:8e6975e5ed6174911a6ff3d60540dfd4844201974602551e10e9e87ab143d81e"
llm = Replicate(
    model=llama2_7b_chat,
    temperature=0.01,
    additional_kwargs={"top_p": 1, "max_new_tokens": 300},
)

# set tokenizer to match LLM
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import ServiceContext

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("YOUR_DATA_DIRECTORY").load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [57]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=[filepath]
)
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")
# documents = SimpleDirectoryReader(filepath).load_data()
# index = VectorStoreIndex.from_documents(documents)

index = VectorStoreIndex.from_documents(
    docs, service_context=service_context
)

Loaded 1 docs


In [58]:
query_engine = index.as_query_engine()
query_engine.query("背隙調整步驟")

ReplicateError: You did not pass an authentication token

In [49]:
query_engine = index.as_query_engine()

result = query_engine.query("背隙調整步驟")
response_text = result.response

print(response_text)

ReplicateError: You did not pass an authentication token